In [1]:
import numpy as np  
import pandas as pd
import re  
import scipy
import nltk
import matplotlib.pyplot as plt


from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
#from pymystem3 import Mystem
#from wordcloud import WordCloud  

from PIL import Image
from scipy.spatial import distance 

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
path = 'C:/Users/v.kozlovskiy/Desktop/DATAS/from_aug24/_ekklesiat_wc_and_other/to_git/'

In [3]:
lemmatizer = WordNetLemmatizer() # lemmatizer for english words

In [4]:
# open prepared file
text_df = pd.read_excel(path + 'text_for_analysis_eng.xlsx')
original = list(text_df['original'])
text = list(text_df['processed'])

In [5]:
def matrix_by_text(text):
    # preprocessing like before making wordcloud
    russian_stopwords = stopwords.words("russian")
    russian_stopwords.extend(['это', 'все', 'вс'])
    english_stopwords = stopwords.words("english")
    union = ' '.join(text)
    text_tokens = word_tokenize(union)
    text_tokens = [token.strip() for token in text_tokens if token not in english_stopwords]
    
    # select unique tokens
    uniq_tokens = pd.Series(text_tokens).unique()

    # matrix for analysis in first dimension strings in text in second unique tokens
    matrix = np.zeros((len(text), len(uniq_tokens)))
    print('Number of strings in text:', len(text))
    print('Number of tokens in text:', len(text_tokens))
    print('Number of unique tokens in text:', len(uniq_tokens))
    print('Matrix shape:', matrix.shape)
    
    # cycle for fill the matrix (if corresponding token is in sentence 1, if no 0)
    for sentence in range(len(text)):
        single_sentence = re.split('[^a-z]', text[sentence])
        #print(single_sentence)
        for word in single_sentence:
            for i in range(len(uniq_tokens)):
                if word == uniq_tokens[i]:
                    #print(word, i)
                    matrix[sentence, i] = 1

    return matrix, uniq_tokens

In [6]:
# function for text filtration and lemmatization of inputed text
def filtration(text):
  text = text.lower()
  text = ''.join(lemmatizer.lemmatize(text))
  text = re.sub(r'[^a-z\']', ' ', text)
  text = text.split()
  print('ok')
  return ' '.join(text)

In [7]:
# function for encoding of inputed text with unique tokens to compare it with matrix
def sentence_encoder(text, uniq_tokens):
    text = filtration(text)
    print(text)
    if len(text) > len(uniq_tokens):
        text = text[:len(uniq_tokens)]
    encoded = np.zeros(len(uniq_tokens))
    for word in text.split():
        for i in range(len(uniq_tokens)):
            if word == uniq_tokens[i]:
                print(word, i)
                encoded[i] = 1
    return encoded, text

In [8]:
# function for counting cousine distance and define closest sentences from text and index or closest sentence
def nearest_phrase(encoded_input, matrix):

    index_list = []  # list for indexes
    distance_list = []  # list for cousine distances
    for i in range(matrix.shape[0]):  # cycle for compare encoded inputed phrase with phrases from file
         index_list.append(i)   
         distance_list.append(scipy.spatial.distance.cosine(encoded_input, matrix[i,:]))    # counting of cousine distance

    # table with results
    distance_table = pd.DataFrame([index_list, distance_list], index=['index', 'distance'])
    distance_table = distance_table.T
    #print(np.argmin(distance_table['distance']))
    needable_index = np.argmin(distance_table['distance'])
    distance_table = distance_table.sort_values(by='distance', ascending=True)
    #print(distance_table.head(5))
    
    return distance_table, needable_index

In [9]:
matrix_a, uniq_tokens = matrix_by_text(text)

Number of strings in text: 222
Number of tokens in text: 2523
Number of unique tokens in text: 793
Matrix shape: (222, 793)


In [10]:
input_text = input()
print('Inputed text:', input_text)

Inputed text: wisdom sun trut


In [11]:
enc_input, phrase = sentence_encoder(input_text, uniq_tokens)
print('Processed inputed text:', phrase)

ok
wisdom sun trut
wisdom 78
sun 14
Processed inputed text: wisdom sun trut


In [12]:
distance_table, needable_index = nearest_phrase(enc_input, matrix_a)
print()
print('5 closest phrase from text:')
print()
for i in distance_table['index'][:5]:
    print(text_df['original'][i], round(distance_table['distance'][i],2))


5 closest phrase from text:

7:11 wisdom is good with an inheritance: and by it there is profit to them that see the sun. 0.42
9:13 this wisdom have i seen also under the sun, and it seemed great unto me. 0.47
4:7 then i returned, and i saw vanity under the sun. 0.65
7:23 all this have i proved by wisdom: i said, i will be wise; but it was far from me. 0.68
1:18 for in much wisdom is much grief: and he that increaseth knowledge increaseth sorrow. 0.71
